# Fruit Classification with CNN (Apples vs Pears)

This notebook trains a CNN to classify fruit images as either apples or pears using PyTorch. It checks for local data first and handles extraction.

## 1. Import Required Libraries

In [ ]:
import os
import gdown
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Write you code here
import pandas as pd

# Load the file, skipping metadata/comment lines
df = pd.read_csv('soc-sign-epinions.txt', sep='\t', comment='#', header=None)

# Manually set column names
df.columns = ['FromNodeId', 'ToNodeId', 'Sign']

# Display the first few rows (optional)
print(df.head())


   FromNodeId  ToNodeId  Sign
0           0         1    -1
1           1    128552    -1
2           2         3     1
3           4         5    -1
4           4       155    -1


In [3]:
! pip install torch torchvision torchaudio
! pip install torch-geometric


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
df.columns

Index(['FromNodeId', 'ToNodeId', 'Sign'], dtype='object')

In [6]:
import torch
from torch_geometric.data import Data

# Get edge_index from FromNodeId and ToNodeId
edge_index = torch.tensor(df[['FromNodeId', 'ToNodeId']].values.T, dtype=torch.long)

# Use Sign column as edge attributes
edge_attr = torch.tensor(df['Sign'].values, dtype=torch.float)

# Dummy node features (one-hot encoded)
x = torch.eye(torch.cat((edge_index[0], edge_index[1])).max().item() + 1)

# Create the PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

print(data)

NameError: name 'df' is not defined

In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch
class SimpleGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [5]:
# num_nodes = torch.cat((edge_index[0], edge_index[1])).max().item() + 1
model = SimpleGCN(input_dim=data.num_node_features, hidden_dim=16, output_dim=2)  # change dims if needed
out = model(data)
print(out)

NameError: name 'data' is not defined

In [ ]:
import torch
import torch.nn.functional as F

# Dummy labels (you would replace this with real node classes)
y = torch.tensor([0, 1, 0, 1, 0, 1], dtype=torch.long)

# Mask to identify which nodes to train on
train_mask = torch.tensor([True, True, True, False, False, False])


In [ ]:
data.y = y
data.train_mask = train_mask


In [ ]:
model = SimpleGCN(input_dim=data.num_node_features, hidden_dim=16, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
print("Predictions:", pred)
